## CMP-3004
## Computer Organization

### Spring 2022


## Housekeeping

- Homework 2
- Project proposal
- Midterm

## Review 

## The Processor 
## Internal Architecture 

General internal architecture and basic function of a processor:

<table><tr>
<td> <img src="cpu_arch.png" alt="Drawing" style="width: 400px;"/> </td>
<td> <img src="cpu_op.png" alt="Drawing" style="width: 250px;"/> </td>
</tr></table>

### Machine Cycle and T-states 

- **Machine cycle:** time-slice during 1-byte (or 1 word) of data are transacted between the processor and some external device (memory or I/O)
- **T-state:** an oscillation of the processor clock

![](./cpu_t.png)

In [ ]:
2 * 10^9 clocks / sec
CPI = 4



### Machine Cycle and T-states 

- The performance of a computer is determined by three key factors: 
    - instruction count
    - clock cycle time
    - clock cycles per instruction (CPI)
- Compiler and the instruction set architecture determine the instruction count required for a given program
- The implementation of the processor determines both the clock cycle time and the number of clock cycles per instruction

### Interrups

Timing, Control and Response example: interrupt

- Interrupt Service Routine (ISR): special program segment with a pre-defined address
    - The processor executes the ISR in response to interrupt (external  async signal)
- The processor cannot leave an instruction’s execution half-way 
- Solution: processors reserve a particular time-slot for checking the existence of any interrupt input signal during the execution of every instruction
    - 8085 reserves the penultimate T-state of the last machine cycle  of any instruction for interrupt signal checking.

## A Basic MIPS Implementation

MIPS is a family of reduced instruction set computer instruction set architectures developed by MIPS Computer Systems

We will be examining an implementation that includes a subset of the core MIPS instruction set:
- The memory-reference instructions load word (lw) and store word (sw)
    - `load 4(edx)`
- The arithmetic-logical instructions add, sub, AND, OR
    - `add edx, eax`
- The instructions branch equal (beq) and jump (j)
    - `jne loop`


In [ ]:
edx + 4

![](./cpu_op.png)

### Instructions compute requirements

- For all these instructions:
    1. Program counter (PC) determines the memory address that contains the instruction to be fetched
        - Since we need the PC to increment, we use the ALU in all instructions
    2. Read one or two registers, using fields of the instruction to select the registers to read. For the load word instruction, only one register is read. Other instructions require reading two registers

- All instruction classes, except jump, use the ALU after reading the registers

    - Memory-reference for an address calculation
    - Arithmetic-logical for the operation execution
    - Branches for comparison

### Instructions compute requirements

- After using the ALU, the actions required to complete various instruction classes differ:

    - **Memory-reference** will need to access the memory to read or write data
    - **Arithmetic-logical** must write the data from the ALU or memory back into a register
    - **Branch instruction** requires to change the next instruction address based on the comparison. Otherwise, PC should be incremented to get the address of the next instruction

### Architecture of the processor

![](./mips2.png)

In [ ]:
add a, b, c
mov c, 4(r)
mov 4(r), a
add a, b, d

In [ ]:
block size = 4
0 - 3
4 - 7
8 - 11

## Datapath

It is a collection of functional units such as ALUs or multipliers, registers, and buses that carry out the fuctional operations

- MIPS’ datapath consist of:
    - **Combinational elements** that operate on data values. No internal storage (e.g. ALU)
        - outputs depend on the inputs
    - **State elements** that contain state in some internal storage (e.g. Registers and Memories)
        - if we disconnect the computer, we can reload the state and carry on


**We are going to assume that registers are 16-bit long, and the memory unit uses 32-bit addresses**

### Clocking methodology

It defines the read/write process of signals

- **Edge-triggered methodology:** any values stored in a sequential element are updated only on a clock edge
- This methodology allows reading the contents of a register, sending the value through some combinational logic, and writing that register in the same clock cycle

![](./mips3.png)

### Building a datapath

If we are going to build our datapath from scratch:
- what elements do we need?
- why do we pick those elements?
- where do we start?

### Building a datapath

What basic datapath elements do we need to fetch instructions from a program?

![](./mips4.png)

To fetch and execute an instruction we need three elements


### R-type instructions


**R-type instructions or arithmetic-logical instructions** read two registers, perform an ALU operation, and write the result to a register

![](./mips5.png)

### Loads and writes

![](./mips6.png)

- the memory unit is a state element with inputs for the address and the write data, and a single output for the read result
- we will need a unit to sign-extend the 16-bit off set field in the instruction to a 32-bit signed value


In [ ]:
reg_size 2       # addr = 2**2 
mem_size 4       # add = 4**2
10 -> 0010

### Branches

Branch on Equal instruction (beq) branches if the two registers are equal. Jump from beq to x

![](./mips7.png)

- uses the ALU to evaluate the branch condition
- a separate adder to compute the branch target as the sum of the incremented PC and the sign-extended, lower 16 bits of the instruction (the branch displacement), shifted left 2 bits
    - the left shift increases the offset by a factor of 4 (e.g., 11 -> 1100)

### Building a single datapath

The datapath for the memory instructions and the R-type instructions

![](./mips8.png)



### Building a simple datapath

The datapath for the main MIPS instructions

![](./mips9.png)

The control unit takes inputs and generates write signals for each datapath element​

## Simple implementation schema

Based on the datapath we designed, we now define a simple instruction set

**ALU Control:** 6 following combinations of four control inputs are needed by all types of instructions

![](./mips10.png)

Remember that the `OPcode` part of the machine instruction determines the instruction that the datapath will execute

    - How do we distinguish between the sum required to load from memory, and the sum of two numbers?

In [ ]:
00 1010
01 0101
10 0010

ALUOP | CONTROL
00
01


In [ ]:
OPCODE | OPERANDS 

### ALU control unit

Small control unit: it receives an extra 2-bit field ALUOp

- The opcode sets the ALUOp bits
- ALUOp = 00 or 01, ignore the function code field
- ALUOp = 10, use the function code field

![](./mips11.png)

### How instructions are structured

![](./mips12.png)

In [ ]:
addr = rs + address

### Distribution to datapath

![](./mips13.png)

### Control signals

The control signals can be set based solely on the opcode field of the instruction (bits 31:26)

![](./mips14.png)

### Datapath with control unit

![](./mips15.png)

### Exercise: 

Read and understand the JUMP example in chapter 4